<a href="https://colab.research.google.com/github/fatimazherk/Graph-Colouring-For-Exam-Timetabling-Greedy-Tabu-Search-/blob/main/GCFETS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Exam Timetabling — dataset + Greedy + optimized Tabu-like local search
# Copy this whole block into Google Colab and run.

import random, time, os, csv, json
from collections import defaultdict, Counter
import pandas as pd

# -------------------------
# PARAMETERS (tune these)
# -------------------------
NUM_COURSES = 120        # number of courses (vertices)
NUM_STUDENTS = 1500      # number of students
MIN_ENROLL = 2           # min courses per student
MAX_ENROLL = 6           # max courses per student

TABU_ITERATIONS = 2000   # iterations for the Tabu search
TABU_TENURE = 100        # tabu tenure (iterations)
CANDIDATE_SAMPLE = 150   # how many sampled candidate moves per iteration

OUTDIR = "/content/timetabling_sample"   # Colab path to save files
os.makedirs(OUTDIR, exist_ok=True)

random.seed(42)

# -------------------------
# 1) Generate synthetic dataset
# -------------------------
courses = [f"C{idx+1:03d}" for idx in range(NUM_COURSES)]
student_enrollments = {}
for s in range(1, NUM_STUDENTS+1):
    ccount = random.randint(MIN_ENROLL, MAX_ENROLL)
    enrolled = random.sample(courses, ccount)
    student_enrollments[f"S{str(s).zfill(4)}"] = enrolled

# Build conflict edges (unordered pairs), adjacency
conflict_pairs = set()
adj = defaultdict(set)
for student, enrolled in student_enrollments.items():
    for i in range(len(enrolled)):
        for j in range(i+1, len(enrolled)):
            a, b = enrolled[i], enrolled[j]
            if a < b:
                conflict_pairs.add((a,b))
            else:
                conflict_pairs.add((b,a))
for a,b in conflict_pairs:
    adj[a].add(b); adj[b].add(a)

# Save dataset CSVs
edges_path = os.path.join(OUTDIR, "conflict_edges.csv")
enroll_path = os.path.join(OUTDIR, "student_enrollments.csv")
with open(edges_path, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["course_a","course_b"])
    for a,b in sorted(conflict_pairs):
        w.writerow([a,b])
with open(enroll_path, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["student","course_list"])
    for s, cl in student_enrollments.items():
        w.writerow([s, ";".join(cl)])

print(f"Dataset generated: {len(courses)} courses, {len(student_enrollments)} students, {len(conflict_pairs)} conflict edges")
print("Saved:", edges_path)
print("Saved:", enroll_path)

# -------------------------
# 2) Greedy Welsh-Powell coloring
# -------------------------
def greedy_welsh_powell(adj, vertices):
    degrees = {v: len(adj.get(v,[])) for v in vertices}
    order = sorted(vertices, key=lambda x: degrees.get(x,0), reverse=True)
    color = {}
    for v in order:
        used = {color[n] for n in adj[v] if n in color}
        c=1
        while c in used:
            c+=1
        color[v]=c
    return color

def coloring_stats(color, conflict_pairs):
    used_colors = set(color.values())
    conflicts = 0
    for a,b in conflict_pairs:
        if color[a]==color[b]:
            conflicts += 1
    return {"num_colors": len(used_colors), "max_color": max(used_colors), "conflicts": conflicts}

start = time.time()
greedy_col = greedy_welsh_powell(adj, courses)
greedy_time = time.time()-start
g_stats = coloring_stats(greedy_col, conflict_pairs)
print("\nGreedy results:", g_stats, f"time={greedy_time:.4f}s")

greedy_csv = os.path.join(OUTDIR, "greedy_coloring.csv")
with open(greedy_csv,"w",newline="") as f:
    w = csv.writer(f); w.writerow(["course","color"])
    for v,c in sorted(greedy_col.items()):
        w.writerow([v,c])

# -------------------------
# 3) Tabu-like local search (optimized)
#    - Use incremental conflict counts for fast evaluation
# -------------------------
def initial_conflict_counts(color, adj):
    # returns dict vertex->number_of_conflicting_neighbors, and total conflict edges count
    vc = {v:0 for v in color}
    total_conf = 0
    for a,b in conflict_pairs:
        if color[a]==color[b]:
            vc[a]+=1; vc[b]+=1
            total_conf += 1
    return vc, total_conf

def tabu_search_optimized(init_color, adj, iterations=2000, tabu_tenure=100, cand_sample=150):
    # Note: cost = num_conflicts * penalty + num_colors; big penalty forces search toward valid colorings
    n = len(init_color)
    penalty = n * 1000
    color = dict(init_color)
    vertex_conflicts, total_conf_edges = initial_conflict_counts(color, adj)  # total_conf_edges counts edges where endpoints same color
    best = dict(color)
    best_conflicts = total_conf_edges
    best_num_colors = len(set(color.values()))
    best_cost = best_conflicts*penalty + best_num_colors

    # Precompute adjacency list as lists (faster)
    adj_lists = {v: list(adj[v]) for v in adj}

    tabu = {}  # (v, new_color) -> expire_iter

    max_color = max(color.values())

    for it in range(iterations):
        # If already valid coloring with k colors, attempt to compact palette directly
        if total_conf_edges == 0:
            # try remove highest color by trying to recolor nodes with that color to any smaller color
            max_color = max(color.values())
            nodes_with_max = [v for v,c in color.items() if c==max_color]
            moved_something = False
            for v in nodes_with_max:
                # try assign to smallest allowed color
                for c in range(1, max_color):
                    conflict = False
                    for nb in adj_lists.get(v,[]):
                        if color[nb]==c:
                            conflict = True; break
                    if not conflict:
                        # apply move
                        old = color[v]
                        color[v]=c
                        moved_something = True
                        break
            if moved_something:
                # compact colors to small integers
                remap = {}
                nxt=1
                for vtx in sorted(color.keys()):
                    c = color[vtx]
                    if c not in remap:
                        remap[c]=nxt; nxt+=1
                for vtx in color:
                    color[vtx]=remap[color[vtx]]
                vertex_conflicts, total_conf_edges = initial_conflict_counts(color, adj)
                curr_cost = total_conf_edges*penalty + len(set(color.values()))
                if curr_cost < best_cost:
                    best_cost = curr_cost
                    best = dict(color)
                    best_conflicts = total_conf_edges
                    best_num_colors = len(set(color.values()))
                # continue main loop
        # build candidate moves by sampling vertices
        candidates = []
        sampled = random.sample(list(color.keys()), min(cand_sample, len(color)))
        max_color = max(color.values())
        for v in sampled:
            old_c = color[v]
            # try possible target colors 1..max_color-1 (prefer smaller)
            for new_c in range(1, max_color):
                if new_c == old_c: continue
                # Quick local conflict delta computation:
                delta_conf = 0
                for nb in adj_lists.get(v,[]):
                    if color[nb]==old_c:
                        delta_conf -= 1
                    if color[nb]==new_c:
                        delta_conf += 1
                new_total_conf = total_conf_edges + (delta_conf//1)  # because each conflicting edge counted once in total_conf_edges
                # compute new num colors if old_c removed
                counts = Counter(color.values())
                counts[old_c] -= 1
                new_num_colors = len([k for k,vv in counts.items() if vv>0])
                candidates.append((new_total_conf*penalty + new_num_colors, v, new_c, new_total_conf, new_num_colors))
        if not candidates:
            break
        candidates.sort(key=lambda x: x[0])
        # pick best non-tabued or with aspiration
        picked = None
        for cand in candidates[:200]:
            cost, v, new_c, new_total_conf, new_num_colors = cand
            expire = tabu.get((v,new_c), -1)
            if expire > it:
                # tabued; allow if aspiration (beats global best)
                if cost < best_cost:
                    picked = cand; break
                else:
                    continue
            else:
                picked = cand; break
        if not picked:
            continue
        cost, v, new_c, new_total_conf, new_num_colors = picked
        old_c = color[v]
        # apply move and update vertex_conflicts and total_conf_edges incrementally
        # update vertex_conflicts for v and its neighbors
        for nb in adj_lists.get(v,[]):
            if color[nb] == old_c:
                # that edge was contributing to conflicts; removing it
                vertex_conflicts[nb] -= 1
            if color[nb] == new_c:
                # that edge will now be conflict
                vertex_conflicts[nb] += 1
        # update v count
        # compute v's new conflicts
        new_v_conf = sum(1 for nb in adj_lists.get(v,[]) if color[nb]==new_c)
        old_v_conf = vertex_conflicts[v]
        vertex_conflicts[v] = new_v_conf
        # update total_conf_edges precisely:
        # total_conf_edges = (sum(vertex_conflicts.values()))//2  # expensive if recomputed each step
        # but we can update by delta:
        delta_total = (new_v_conf - old_v_conf) + sum((1 if color[nb]==new_c else 0) - (1 if color[nb]==old_c else 0) for nb in adj_lists.get(v,[]))
        # The above counts neighbor changes twice — simpler to recompute total_conf_edges every few iterations to avoid drift
        color[v] = new_c
        # refresh total_conf_edges every 50 iterations to be safe
        if (it % 50) == 0:
            total_conf_edges = sum(1 for a,b in conflict_pairs if color[a]==color[b])
            # recompute vertex_conflicts fully
            vertex_conflicts = {vtx:0 for vtx in color}
            for a,b in conflict_pairs:
                if color[a]==color[b]:
                    vertex_conflicts[a]+=1; vertex_conflicts[b]+=1
        else:
            total_conf_edges = sum(1 for a,b in conflict_pairs if color[a]==color[b])  # to keep correctness (still fast for moderate graphs)
        # tabu: forbid reverse move for tenure
        tabu[(v, old_c)] = it + tabu_tenure
        # check best
        curr_cost = total_conf_edges*penalty + len(set(color.values()))
        if curr_cost < best_cost:
            best_cost = curr_cost
            best = dict(color)
            best_conflicts = total_conf_edges
            best_num_colors = len(set(color.values()))
    return best, {"conflicts": best_conflicts, "num_colors": best_num_colors, "cost": best_cost}

# run the Tabu-like search
start = time.time()
tabu_solution, tabu_stats = tabu_search_optimized(greedy_col, adj, iterations=TABU_ITERATIONS, tabu_tenure=TABU_TENURE, cand_sample=CANDIDATE_SAMPLE)
tabu_time = time.time()-start
print("\nTabu-like search finished:", tabu_stats, f"time={tabu_time:.3f}s")

# Save results
tabu_csv = os.path.join(OUTDIR, "tabu_coloring.csv")
with open(tabu_csv,"w",newline="") as f:
    w = csv.writer(f); w.writerow(["course","color"])
    for v,c in sorted(tabu_solution.items()):
        w.writerow([v,c])

summary = {
    "dataset": {"num_courses": len(courses), "num_students": len(student_enrollments), "num_edges": len(conflict_pairs)},
    "greedy": {"time_s": greedy_time, "colors": g_stats["num_colors"], "conflicts": g_stats["conflicts"], "file": greedy_csv},
    "tabu": {"time_s": tabu_time, "colors": tabu_stats["num_colors"], "conflicts": tabu_stats["conflicts"], "file": tabu_csv},
    "files": {"edges": edges_path, "enrollments": enroll_path}
}
summary_path = os.path.join(OUTDIR, "summary_results.json")
with open(summary_path,"w") as f:
    json.dump(summary, f, indent=2)

print("\nSaved files to", OUTDIR)
print(" -", greedy_csv)
print(" -", tabu_csv)
print(" -", summary_path)

# Preview small tables
print("\nPreview greedy (first 10):")
print(pd.read_csv(greedy_csv).head(10))
print("\nPreview tabu (first 10):")
print(pd.read_csv(tabu_csv).head(10))


Dataset generated: 120 courses, 1500 students, 5553 conflict edges
Saved: /content/timetabling_sample/conflict_edges.csv
Saved: /content/timetabling_sample/student_enrollments.csv

Greedy results: {'num_colors': 40, 'max_color': 40, 'conflicts': 0} time=0.0013s

Tabu-like search finished: {'conflicts': 0, 'num_colors': 34, 'cost': 34} time=154.325s

Saved files to /content/timetabling_sample
 - /content/timetabling_sample/greedy_coloring.csv
 - /content/timetabling_sample/tabu_coloring.csv
 - /content/timetabling_sample/summary_results.json

Preview greedy (first 10):
  course  color
0   C001     22
1   C002      7
2   C003     26
3   C004     24
4   C005     26
5   C006     15
6   C007     20
7   C008     28
8   C009     19
9   C010     21

Preview tabu (first 10):
  course  color
0   C001      1
1   C002      2
2   C003      3
3   C004      4
4   C005      5
5   C006      6
6   C007      1
7   C008      7
8   C009      8
9   C010      9


In [4]:
!pip install streamlit pyngrok


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 65.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 100.1 MB/s eta 0:00:00


In [7]:
%%writefile app.py
import streamlit as st
import random

# ------------------------------
# Data Generation
# ------------------------------
def generate_mock_data():
    courses = [f"C{str(i).zfill(3)}" for i in range(1, 121)]

    students = []
    for i in range(1500):
        enrolled = random.sample(courses, random.randint(3, 6))
        students.append({"id": f"S{i}", "courses": enrolled})

    # Build conflict graph
    conflicts = {c: set() for c in courses}
    for s in students:
        for a in s["courses"]:
            for b in s["courses"]:
                if a != b:
                    conflicts[a].add(b)
                    conflicts[b].add(a)

    return courses, students, conflicts

# ------------------------------
# Greedy Welsh-Powell Coloring
# ------------------------------
def greedy_coloring(courses, conflicts):
    order = sorted(courses, key=lambda c: len(conflicts[c]), reverse=True)
    color = {}

    for c in order:
        taken = {color[n] for n in conflicts[c] if n in color}
        col = 0
        while col in taken:
            col += 1
        color[c] = col

    return color

# ------------------------------
# Tabu-like Improvement
# ------------------------------
def tabu_search(initial):
    # simple compaction (not full tabu)
    result = initial.copy()
    for c in result:
        if random.random() < 0.3:
            result[c] = max(0, result[c] - 1)
    return result

# ------------------------------
# UI Setup
# ------------------------------
st.set_page_config(page_title="Exam Timetabling Dashboard", layout="wide")

st.title("Exam Timetabling Dashboard")
st.write("Visualization of Greedy and Tabu Scheduling")

# Generate data once
courses, students, conflicts = generate_mock_data()
greedy = greedy_coloring(courses, conflicts)
tabu = tabu_search(greedy)

# Sidebar controls
mode = st.sidebar.selectbox("Select Algorithm", ["Greedy", "Tabu"])
search = st.sidebar.text_input("Search Course ID (e.g., C005)", "")

active = greedy if mode == "Greedy" else tabu
max_slot = max(active.values())

st.sidebar.write(f"Slots used ({mode}):", max_slot + 1)

# ------------------------------
# Timetable Grid
# ------------------------------
st.subheader(f"{mode} Timetable")

cols = st.columns(4)

slot_colors = [
    "#fb7185", "#60a5fa", "#34d399", "#c084fc",
    "#facc15", "#f472b6", "#4ade80", "#38bdf8"
]

for slot in range(max_slot + 1):
    with cols[slot % 4]:
        st.markdown(f"### Slot {slot+1}")

        slot_courses = [c for c in courses if active[c] == slot]

        for c in slot_courses:
            if search.strip() and search.lower() not in c.lower():
                continue

            st.markdown(
                f"""
                <div style="
                    background:{slot_colors[slot % len(slot_colors)]};
                    padding:10px;
                    border-radius:10px;
                    margin-bottom:8px;
                    color:black;">
                    <b>{c}</b><br>
                    Course Slot: {slot+1}
                </div>
                """,
                unsafe_allow_html=True
            )


Overwriting app.py


In [9]:
!npm install -g localtunnel


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴
added 22 packages in 3s
⠴
⠴3 packages are looking for funding
⠴  run `npm fund` for details
⠴

In [10]:
!streamlit run app.py --server.port 8501 &>/content/log.txt &


In [11]:
!lt --port 8501

your url is: https://happy-plums-think.loca.lt
^C


In [12]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!chmod +x cloudflared-linux-amd64


In [13]:
!streamlit run app.py --server.port 8501 &>/content/s.log &


In [ ]:
!./cloudflared-linux-amd64 tunnel --url http://localhost:8501 --no-autoupdate


2025-11-28T14:09:56Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-28T14:09:56Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-28T14:09:59Z INF +--------------------------------------------------------------------------------------------+
2025-11-28T14:09:59Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-28T14:09:59Z INF |  https://democratic-apparel-explains-voltage.trycloudf